In [1]:
#pip install pandas numpy==1.23.5 tensorflow-gpu==2.10.1 scikit-learn tqdm 

In [6]:
#conda install -c conda-forge cudatoolkit=11.2 cudnn=8.1.0

In [7]:
import tensorflow as tf
print("Số lượng GPU khả dụng: ", len(tf.config.list_logical_devices('GPU')))

Số lượng GPU khả dụng:  1


### Inference

In [82]:
import os
import json
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Conv1D
from sklearn.metrics import confusion_matrix
from metrics import compute_metrics # Hàm của bạn

# --- CẤU HÌNH ĐƯỜNG DẪN ---
MODEL_PATH = r'D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\MMSplice\pretrained_model' # Thư mục chứa donor.h5 và acceptor.h5
DATA_FOLDER = r'D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val'
RESULTS_FOLDER = 'results'
TEST_FILES = ['test_1_1_1.csv', 'test_2_1_1.csv', 'test_4_1_1.csv', 'test_10_1_1.csv', 'test_data.csv']

os.makedirs(RESULTS_FOLDER, exist_ok=True)

# Tự định nghĩa lại lớp ConvDNA để xử lý các tham số lạ từ file .h5
class ConvDNA(Conv1D):
    def __init__(self, *args, **kwargs):
        # Lấy seq_length ra nếu có, nếu không thì bỏ qua 
        # (pop giúp loại bỏ nó khỏi kwargs để không gây lỗi cho Conv1D)
        self.seq_length = kwargs.pop('seq_length', None)
        
        # Nếu mô hình còn các tham số "lạ" khác, bạn có thể pop tương tự
        # Hoặc dùng cách an toàn hơn bên dưới:
        super(ConvDNA, self).__init__(*args, **kwargs)

    def get_config(self):
        config = super(ConvDNA, self).get_config()
        config.update({'seq_length': self.seq_length})
        return config

# Bây giờ truyền nó vào custom_objects
custom_objs = {'ConvDNA': ConvDNA}

# --- LOAD MODELS TRỰC TIẾP ---
print("Đang load models từ file .h5...")
donor_model = tf.keras.models.load_model(
        os.path.join(MODEL_PATH, 'donor.h5'), 
        compile=False, 
        custom_objects=custom_objs
    )
print("Load donor thành công!")

acceptor_model = tf.keras.models.load_model(
    os.path.join(MODEL_PATH, 'acceptor.h5'), 
    compile=False, 
    custom_objects=custom_objs
)
print("Load acceptor thành công!")

# --- CÁC HÀM XỬ LÝ CHUỖI ---
def reverse_complement(seq):
    complement = {'A': 'T', 'C': 'G', 'G': 'C', 'T': 'A', 'N': 'N'}
    return "".join(complement.get(base, base) for base in reversed(seq.upper()))

def one_hot_dna(seq):
    mapping = {'A': 0, 'C': 1, 'G': 2, 'T': 3}
    arr = np.zeros((len(seq), 4), dtype=np.float32)
    for i, base in enumerate(seq.upper()):
        if base in mapping:
            arr[i, mapping[base]] = 1.0
    return arr

# --- PIPELINE INFERENCE ---
def run_pipeline(file_name):
    df = pd.read_csv(os.path.join(DATA_FOLDER, file_name))
    y_true = df['Splicing_types'].values
    
    d_inputs, a_inputs = [], []
    
    for _, row in df.iterrows():
        seq = row['sequence']
        mid = len(seq) // 2
        # Donor (9bp): 3bp Exon | 6bp Intron
        d_inputs.append(one_hot_dna(seq[mid-5 : mid+13]))
        # Acceptor (50bp): 40bp Intron | 10bp Exon
        a_inputs.append(one_hot_dna(seq[mid-49 : mid+4]))

    # Predict Batch
    d_scores = donor_model.predict(np.stack(d_inputs), batch_size=512, verbose=0).flatten()
    a_scores = acceptor_model.predict(np.stack(a_inputs), batch_size=512, verbose=0).flatten()

    print(f"Donor scores: Min={d_scores.min():.4f}, Max={d_scores.max():.4f}, Mean={d_scores.mean():.4f}")
    print(f"Acceptor scores: Min={a_scores.min():.4f}, Max={a_scores.max():.4f}, Mean={a_scores.mean():.4f}")
    
    # 1. Tạo ma trận xác suất (Sử dụng Normalization thay vì Softmax)
    y_probs = np.zeros((len(d_scores), 3))
    y_probs[:, 1] = d_scores
    y_probs[:, 2] = a_scores
    y_probs[:, 0] = 1 - np.maximum(d_scores, a_scores)
    
    # Chuẩn hóa để tổng mỗi hàng = 1 cho AUC metrics
    row_sums = y_probs.sum(axis=1, keepdims=True)
    y_probs = y_probs / np.where(row_sums == 0, 1e-10, row_sums)
    
    # 2. Tạo nhãn dự đoán (y_pred) 
    # Nếu bạn muốn dùng Threshold để ưu tiên lớp hiếm (Donor/Acceptor)
    THRESHOLD = 0.48
    y_pred = []
    for d, a in zip(d_scores, a_scores):
        if d > THRESHOLD and d > a: 
            y_pred.append(1)
        elif a > THRESHOLD and a > d: 
            y_pred.append(2)
        else: 
            y_pred.append(0)
            
    return y_true, np.array(y_pred), y_probs

# --- CHẠY VÀ LƯU KẾT QUẢ ---
for f in TEST_FILES:
    print(f"Processing {f}...")
    y_true, y_pred, y_probs = run_pipeline(f)
    
    res = {
        "metrics": compute_metrics(y_true, y_pred, y_probs),
        "confusion_matrix": confusion_matrix(y_true, y_pred, labels=[0, 1, 2]).tolist()
    }
    
    with open(os.path.join(RESULTS_FOLDER, f.replace('.csv', '_results.json')), 'w') as out:
        json.dump(res, out, indent=4)

print("Xong!")

Đang load models từ file .h5...
Load donor thành công!
Load acceptor thành công!
Processing test_1_1_1.csv...
Donor scores: Min=0.0006, Max=0.9998, Mean=0.3565
Acceptor scores: Min=0.0000, Max=0.9999, Mean=0.3057
Processing test_2_1_1.csv...
Donor scores: Min=0.0006, Max=0.9998, Mean=0.2997
Acceptor scores: Min=0.0000, Max=0.9999, Mean=0.2496
Processing test_4_1_1.csv...
Donor scores: Min=0.0006, Max=0.9998, Mean=0.2438
Acceptor scores: Min=0.0000, Max=0.9999, Mean=0.1922
Processing test_10_1_1.csv...
Donor scores: Min=0.0006, Max=0.9998, Mean=0.1880
Acceptor scores: Min=0.0000, Max=0.9999, Mean=0.1361
Processing test_data.csv...
Donor scores: Min=0.0004, Max=0.9998, Mean=0.1379
Acceptor scores: Min=0.0000, Max=0.9999, Mean=0.0857
Xong!


In [50]:
def check_motifs(df, num_samples=10):
    mid = len(df.iloc[0]['sequence']) // 2
    print("--- Kiểm tra Motif Donor (Cần GT ở vị trí cắt) ---")
    donors = df[df['Splicing_types'] == 1].sample(num_samples)
    for _, row in donors.iterrows():
        # Giả sử mid là vị trí bắt đầu Intron
        seq = row['sequence']
        # In ra 2 base tại vị trí cắt
        print(f"Sequence: ...{seq[mid-2:mid]} | {seq[mid:mid+2]}... -> Motif: {seq[mid:mid+2]}")

df = pd.read_csv(r'D:\my_project\Bio_paper\Bio_sequence_Research_AITALAB\train\task1_splicing_prediction\data_preparation\train_val\test_1_1_1.csv')
check_motifs(df)

--- Kiểm tra Motif Donor (Cần GT ở vị trí cắt) ---
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...GG | GC... -> Motif: GC
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...GT | GT... -> Motif: GT
Sequence: ...AG | GT... -> Motif: GT
Sequence: ...AG | GT... -> Motif: GT


In [55]:
def find_best_offset(df, model, label_type):
    results = []
    # Chỉ lấy các mẫu chuẩn (True Label) để test
    test_df = df[df['Splicing_types'] == label_type]
    
    for offset in range(-5, 6): # Thử từ 295 đến 305
        mid = 300 + offset
        inputs = []
        for seq in test_df['sequence']:
            if label_type == 1: # Donor
                inputs.append(one_hot_dna(seq[mid-3 : mid+15]))
            else: # Acceptor
                inputs.append(one_hot_dna(seq[mid-50 : mid+3]))
        
        preds = model.predict(np.array(inputs), verbose=0)
        avg_score = np.mean(preds)
        results.append((offset, avg_score))
        print(f"Offset {offset} (Mid={mid}): Avg Score = {avg_score:.4f}")
    
    best_offset = max(results, key=lambda x: x[1])
    return best_offset

# Chạy thử cho Donor
print("--- Đang tìm vị trí vàng cho Donor ---")
best_d = find_best_offset(df, donor_model, 1)
print(f"==> Donor nên cắt tại mid = 300 + ({best_d[0]})")

--- Đang tìm vị trí vàng cho Donor ---
Offset -5 (Mid=295): Avg Score = 0.0676
Offset -4 (Mid=296): Avg Score = 0.0594
Offset -3 (Mid=297): Avg Score = 0.2209
Offset -2 (Mid=298): Avg Score = 0.8679
Offset -1 (Mid=299): Avg Score = 0.0237
Offset 0 (Mid=300): Avg Score = 0.0281
Offset 1 (Mid=301): Avg Score = 0.0352
Offset 2 (Mid=302): Avg Score = 0.0754
Offset 3 (Mid=303): Avg Score = 0.0607
Offset 4 (Mid=304): Avg Score = 0.0436
Offset 5 (Mid=305): Avg Score = 0.0554
==> Donor nên cắt tại mid = 300 + (-2)


In [58]:
def find_best_offset(df, model, label_type):
    results = []
    # Chỉ lấy các mẫu chuẩn (True Label) để test
    test_df = df[df['Splicing_types'] == label_type]
    
    for offset in range(-5, 6): # Thử từ 295 đến 305
        mid = 300 + offset
        inputs = []
        for seq in test_df['sequence']:
            if label_type == 1: # Donor
                inputs.append(one_hot_dna(seq[mid-3 : mid+15]))
            else: # Acceptor
                inputs.append(one_hot_dna(seq[mid-50 : mid+3]))
        
        preds = model.predict(np.array(inputs), verbose=0)
        avg_score = np.mean(preds)
        results.append((offset, avg_score))
        print(f"Offset {offset} (Mid={mid}): Avg Score = {avg_score:.4f}")
    
    best_offset = max(results, key=lambda x: x[1])
    return best_offset

# Chạy thử cho Acceptor
print("--- Đang tìm vị trí vàng cho Acceptor ---")
best_d = find_best_offset(df, acceptor_model, 2)
print(f"==> Acceptor nên cắt tại mid = 300 + ({best_d[0]})")

--- Đang tìm vị trí vàng cho Acceptor ---
Offset -5 (Mid=295): Avg Score = 0.1180
Offset -4 (Mid=296): Avg Score = 0.1109
Offset -3 (Mid=297): Avg Score = 0.1268
Offset -2 (Mid=298): Avg Score = 0.1745
Offset -1 (Mid=299): Avg Score = 0.1912
Offset 0 (Mid=300): Avg Score = 0.2024
Offset 1 (Mid=301): Avg Score = 0.8210
Offset 2 (Mid=302): Avg Score = 0.0711
Offset 3 (Mid=303): Avg Score = 0.0530
Offset 4 (Mid=304): Avg Score = 0.1062
Offset 5 (Mid=305): Avg Score = 0.0856
==> Acceptor nên cắt tại mid = 300 + (1)


In [18]:
print("Cấu trúc Donor Model:")
donor_model.summary()

print("\nCấu trúc Acceptor Model:")
acceptor_model.summary()

Cấu trúc Donor Model:
Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 ss (InputLayer)             [(None, 18, 4)]           0         
                                                                 
 flatten_1 (Flatten)         (None, 72)                0         
                                                                 
 dense1 (Dense)              (None, 128)               9344      
                                                                 
 batch_normalization_1 (Batc  (None, 128)              512       
 hNormalization)                                                 
                                                                 
 re_lu_1 (ReLU)              (None, 128)               0         
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                     